In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from package.utils.data_loder import PDFLoader
from package.interface import SourceOptions
from package.flows.offline import OfflineFlow

d:\broai-arai\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = PDFLoader(
    source=SourceOptions(path="./sources/storm.pdf", type="pdf")
)
contexts = loader.run()

Markdown headings: max(2)


d:\broai-arai\backend\package\utils\data_loder.py:22: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(text)
d:\broai-arai\backend\package\utils\data_loder.py:23: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)
d:\broai-arai\backend\package\utils\data_loder.py:24: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)
d:\broai-arai\backend\package\utils\data_loder.py:30: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts, max_tokens=max_tokens, overlap=overlap)


In [ ]:
olf = OfflineFlow()

In [14]:
import torch
torch.cuda.is_available()

True